In [ ]:
 %run "Path to the setup.ipynb file"

importing standardlib modules
importing google drive
importing tensorflow/keras modules
importing miscelaneaous modules
       utterance_ID                                               text            speaker  emotion  conversation_ID  global_ID
11658             1                    My best shoes , so good to me .             Leslie      joy             1182      11659
11659             2                             I wear them everyday .             Leslie      joy             1182      11660
11660             3             Down at the heel , holes in the toes .             Leslie      joy             1182      11661
11661             4                      Do not care what people say .             Leslie      joy             1182      11662
11662             5           My feet best friends , pals to the end .             Leslie      joy             1182      11663
11663             6                    With them I am one hot chicky .             Leslie      joy             1182   

# Models

In [ ]:
# @title Simple Fully Connected NN

def get_nn(input_shape, output_shape, layers: Iterable, dropout=0.3):
    sequence_input = Input(shape=input_shape, dtype='float32')

    # First layer is connected to input
    dense_block = Dense(layers[0], activation='relu')(sequence_input)
    dense_block = Dropout(dropout)(dense_block)

    # Add additional layers
    for layer in layers[1:]:
        dense_block = Dense(layer, activation='relu')(dense_block)
        dense_block = Dropout(dropout)(dense_block)

    # Add output layer
    l_flat = Flatten()(dense_block)
    predictions = Dense(output_shape, activation='softmax')(l_flat)

    return Model(sequence_input, predictions)

In [ ]:
#  @title Deep CNN

def get_cnn(input_shape, output_shape, filters: int, kernels: Iterable, pool_size=2, dropout=0.3):
    sequence_input = Input(shape=input_shape, dtype='float32')

    # First CNN layer is connected to input
    conv_block = Conv1D(filters, kernels[0], activation='relu')(sequence_input)
    conv_block = MaxPooling1D(pool_size)(conv_block)
    conv_block = Dropout(dropout)(conv_block)

    # Add additional CNN layers
    for kernel in kernels[1:]:
        conv_block = Conv1D(filters, kernel, activation='relu')(conv_block)
        conv_block = MaxPooling1D(pool_size)(conv_block)
        conv_block = Dropout(dropout)(conv_block)

    # Add dense layers
    l_flat = Flatten()(conv_block)
    l_dense = Dense(128, activation='relu')(l_flat)
    predictions = Dense(output_shape, activation='softmax')(l_dense)

    return Model(sequence_input, predictions)

In [ ]:
# @title Parallel CNN

def get_parallel_cnn(input_shape, output_shape, filters: int, kernels: Iterable, pool_size=4, dropout=0.3, l2=0.0001) -> Model:
  sequence_input = Input(shape=input_shape, dtype='float32')


  conv_layers = list()
  for kernel in kernels:
    new_conv = Conv1D(filters=filters,
                      kernel_size=kernel,
                      activation='relu',
                      kernel_regularizer=regularizers.l2(l2),
                      name=f"{kernel}k_conv1d"
                      )(sequence_input)
    new_conv = Dropout(dropout)(new_conv)
    conv_layers.append(new_conv)

  l_merge = Concatenate(axis=1)(conv_layers)
  l_pool = MaxPooling1D(pool_size)(l_merge)
  l_drop = Dropout(dropout)(l_pool)
  l_flat = Flatten()(l_drop)
  l_dense = Dense(128, activation='relu')(l_flat)
  predictions = Dense(output_shape, activation='softmax')(l_dense)
  return Model(sequence_input, predictions)

# Analysis

In [ ]:
def plot_training_results(model_log):
    accuracy = model_log.history['acc']
    val_accuracy = model_log.history['val_acc']
    loss = model_log.history['loss']
    val_loss = model_log.history['val_loss']

    epochs = range(1, len(accuracy)+1)

    plt.plot(epochs, accuracy, 'g', label='Training accuracy')
    plt.plot(epochs, val_accuracy, 'r', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.legend()

    plt.figure()

    plt.plot(epochs, loss, 'g', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

    plt.show()

In [ ]:
def plot_confusion_matrix(cm, labels, normalize=True, title='Confusion Matrix Prep (Validation Set)', cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels, rotation=45)
    plt.yticks(tick_marks, labels)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('TRUE LABEL')
    plt.xlabel('PPREDICTED LABEL')

# Training

In [ ]:
# @title load data (takes a while, don't rerun unnecessarily)
sample, target = load_batch_utterances(conv_df, EMBEDDINGS_PATH, batch_size=8192)
sample_val, target_val = load_batch_utterances(conv_df, EMBEDDINGS_PATH, batch_size=1024, validation=True)

  0%|          | 0/8192 [00:00<?, ?it/s]

  0%|          | 0/1024 [00:00<?, ?it/s]

Requested batch size exceeds available data, max available 733


In [ ]:
LR = 5e-5
DROPOUT = 0.3
L2 = 0.0001
EPOCHS = 10
BATCH_SIZE = 16

In [ ]:
# @title train and evaluate simple FC NN model

model = get_nn(layers=[512,256,128,64],
                input_shape=(data_params['len_utterance'], data_params['len_embedding']),
                output_shape=data_params['n_emotions'],
                dropout=DROPOUT)
adam = optimizers.Adam(learning_rate=LR)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['acc'])
plot_model(model)

model_log = model.fit(sample, target,
                      validation_data=(sample_val, target_val),
                      epochs=EPOCHS,
                      batch_size=BATCH_SIZE)

plot_training_results(model_log)
model.save("/content/drive/MyDrive/bachelor_thesis/models/emo_model_mlp.keras")

Epoch 1/10
512/512 [==============================] - 7s 5ms/step - loss: 1.6025 - acc: 0.4294 - val_loss: 1.4594 - val_acc: 0.4727
Epoch 2/10
512/512 [==============================] - 2s 4ms/step - loss: 1.3981 - acc: 0.5094 - val_loss: 1.3321 - val_acc: 0.5293
Epoch 3/10
512/512 [==============================] - 2s 4ms/step - loss: 1.2989 - acc: 0.5477 - val_loss: 1.2759 - val_acc: 0.5674
Epoch 4/10
512/512 [==============================] - 2s 5ms/step - loss: 1.2426 - acc: 0.5775 - val_loss: 1.2349 - val_acc: 0.5801
Epoch 5/10
512/512 [==============================] - 2s 4ms/step - loss: 1.1995 - acc: 0.5955 - val_loss: 1.2097 - val_acc: 0.5781
Epoch 6/10
512/512 [==============================] - 2s 4ms/step - loss: 1.1599 - acc: 0.6073 - val_loss: 1.1740 - val_acc: 0.5967
Epoch 7/10
512/512 [==============================] - 2s 4ms/step - loss: 1.1254 - acc: 0.6168 - val_loss: 1.1751 - val_acc: 0.5938
Epoch 8/10
512/512 [==============================] - 2s 4ms/step - loss: 1.

In [ ]:
# @title train and evaluate a CNN model

model = get_cnn(filters=24,
                kernels=[8, 5, 2],
                input_shape=(data_params['len_utterance'], data_params['len_embedding']),
                output_shape=data_params['n_emotions'],
                dropout=DROPOUT)
adam = optimizers.Adam(learning_rate=LR)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['acc'])
plot_model(model)

model_log = model.fit(sample, target,
                      validation_data=(sample_val, target_val),
                      epochs=15,
                      batch_size=32,
                      verbose=1)

plot_training_results(model_log)
model.save("/content/drive/MyDrive/bachelor_thesis/models/emo_model_cnn.keras")


Epoch 1/15
256/256 [==============================] - 6s 8ms/step - loss: 1.6600 - acc: 0.4099 - val_loss: 1.6004 - val_acc: 0.3915
Epoch 2/15
256/256 [==============================] - 2s 6ms/step - loss: 1.5105 - acc: 0.4580 - val_loss: 1.5168 - val_acc: 0.4775
Epoch 3/15
256/256 [==============================] - 1s 6ms/step - loss: 1.4366 - acc: 0.4840 - val_loss: 1.4549 - val_acc: 0.4898
Epoch 4/15
256/256 [==============================] - 1s 6ms/step - loss: 1.3938 - acc: 0.4950 - val_loss: 1.4152 - val_acc: 0.5198
Epoch 5/15
256/256 [==============================] - 1s 6ms/step - loss: 1.3684 - acc: 0.5073 - val_loss: 1.3858 - val_acc: 0.5171
Epoch 6/15
256/256 [==============================] - 1s 6ms/step - loss: 1.3365 - acc: 0.5231 - val_loss: 1.3707 - val_acc: 0.5402
Epoch 7/15
256/256 [==============================] - 1s 6ms/step - loss: 1.3148 - acc: 0.5371 - val_loss: 1.3357 - val_acc: 0.5416
Epoch 8/15
256/256 [==============================] - 1s 6ms/step - loss: 1.